<b><font size="5">Segmenting and Clustering of Toronto Neighborhood</font></b>

<b>1. Import Libraries </b>

In [1]:
#import libraries
import pandas as pd
import numpy as np

# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#import folium to render map
!conda install -c conda-forge folium=0.5.0 --yes
import folium

print("Imported!")

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  54.10 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  35.12 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  39.68 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  47.67 MB/s
Imported!


In [2]:
#Install beautifulsoup and html parser
!easy_install beautifulsoup4
!easy_install html5lib

Searching for beautifulsoup4
Best match: beautifulsoup4 4.6.0
Adding beautifulsoup4 4.6.0 to easy-install.pth file

Using /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Processing dependencies for beautifulsoup4
Finished processing dependencies for beautifulsoup4
Searching for html5lib
Best match: html5lib 0.999999999
Adding html5lib 0.999999999 to easy-install.pth file

Using /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Processing dependencies for html5lib
Finished processing dependencies for html5lib


<b>2. Get the web page contents from the Wikipedia web site</b>

In [3]:
#Get page contents from the given URL
import requests
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
#print(page.content)


<b>3. Use the HTML parser to extract table from the web page</b>

In [4]:
#Using BeautifulSoup to extract the page
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

<p>
First, get contents in tags 'tr' from the table.
Next, extract the contents within the header tags 'th'
Last, extract the data within tags 'td' if Borough is not 'Not assigned'.
Change the value of Neighbourhood if its value is 'Not assigned'.
Save the data to a dataframe.
</p>

In [5]:
#Extract table from the page
html = list(soup.children)[2]
list(html.children)
body = list(html.children)[3]
body
canada_table = soup.find_all('table')[0]

list_tr = canada_table.find_all("tr")

#Extract header tags
header = []
tr = list_tr[0]
for th in tr:
    if(th != "\n"):
        header.append(th.get_text().rstrip())

#Read table contents and change Borough to Postcode if Borough is 'Not assigned'
records = []
for tr in list_tr:
    trs = tr.findAll("td")
    if trs:
        record = []
        record.append(trs[0].text)
        #Check if Borough is 'Not assigned'
        if(trs[1].text != "Not assigned"):
            record.append(trs[1].text)
            #Check if 'Neighbourhood if 'Not assigned'
            if(trs[2].text.rstrip() == "Not assigned"):
                record.append(trs[1].text)
            else:
                record.append(trs[2].text.rstrip())
            records.append(record)
    
#Create dataframe with table contents
canada_df = pd.DataFrame(data=records, columns = header)
print(canada_df.shape)
#print(canada_df)


(212, 3)


In [6]:
#Find rows with same Postcode and Borough and concat the Neighbourhood names 
#Create a new dataframe with these changes
list_canada = []
for (k1, k2), group in canada_df.groupby(['Postcode', 'Borough']):
    col = group['Neighbourhood']
    names = pd.Series(group['Neighbourhood'].values)
    grouped_names = names.str.cat(sep=',')
    new_row = [k1, k2, grouped_names]
    list_canada.append(new_row)

new_canada_df=pd.DataFrame(list_canada,columns=['Postcode','Borough', 'Neighbourhood'])
print(new_canada_df.shape)
print(new_canada_df.head(40))

(103, 3)
   Postcode       Borough                                      Neighbourhood
0       M1B   Scarborough                                      Rouge,Malvern
1       M1C   Scarborough               Highland Creek,Rouge Hill,Port Union
2       M1E   Scarborough                    Guildwood,Morningside,West Hill
3       M1G   Scarborough                                             Woburn
4       M1H   Scarborough                                          Cedarbrae
5       M1J   Scarborough                                Scarborough Village
6       M1K   Scarborough          East Birchmount Park,Ionview,Kennedy Park
7       M1L   Scarborough                      Clairlea,Golden Mile,Oakridge
8       M1M   Scarborough      Cliffcrest,Cliffside,Scarborough Village West
9       M1N   Scarborough                         Birch Cliff,Cliffside West
10      M1P   Scarborough  Dorset Park,Scarborough Town Centre,Wexford He...
11      M1R   Scarborough                                   Maryval

<b>4. Read CSV file with Geospatial data of Canada to a dataframe and merge with the Canada data</b>

In [7]:
#Read the csv file that has Latitude and Longitude data to a DataFrame
lat_lon_data = pd.read_csv('http://cocl.us/Geospatial_data')
lat_lon_data.columns = ['Postcode', 'Latitude', 'Longitude']
lat_lon_data.head(5)


,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
#Merge canada dataframe with the Latitude and Longitude data
all_canada_df = pd.merge(new_canada_df, lat_lon_data, on = ['Postcode'])
all_canada_df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


<b>5. Extracting only Toronto data from the Canada data</b>

In [16]:
#Create a dataframe with rows that have Toronto in Borough
list_toronto = []
toronto_df = pd.DataFrame(columns = ['Postcode','Borough', 'Neighbourhood', 'Latitude', 'Longitude'])

for (k1), group in all_canada_df.groupby(['Borough']):
    if 'Toronto' in k1:
        toronto_df = toronto_df.append(group)

toronto_df.reset_index(drop=True, inplace=True)

print(toronto_df.shape)
print(toronto_df.head(38))

                

(38, 5)
   Postcode           Borough  \
0       M4N   Central Toronto   
1       M4P   Central Toronto   
2       M4R   Central Toronto   
3       M4S   Central Toronto   
4       M4T   Central Toronto   
5       M4V   Central Toronto   
6       M5N   Central Toronto   
7       M5P   Central Toronto   
8       M5R   Central Toronto   
9       M4W  Downtown Toronto   
10      M4X  Downtown Toronto   
11      M4Y  Downtown Toronto   
12      M5A  Downtown Toronto   
13      M5B  Downtown Toronto   
14      M5C  Downtown Toronto   
15      M5E  Downtown Toronto   
16      M5G  Downtown Toronto   
17      M5H  Downtown Toronto   
18      M5J  Downtown Toronto   
19      M5K  Downtown Toronto   
20      M5L  Downtown Toronto   
21      M5S  Downtown Toronto   
22      M5T  Downtown Toronto   
23      M5V  Downtown Toronto   
24      M5W  Downtown Toronto   
25      M5X  Downtown Toronto   
26      M6G  Downtown Toronto   
27      M4E      East Toronto   
28      M4K      East Toronto   
29

In [13]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_df['Borough'].unique()),
        toronto_df.shape[0]
    )
)

The dataframe has 4 boroughs and 38 neighborhoods.


<b>6. Get Geographical coordinates of Toronto and render a map with Neighbourhoods</b>

In [17]:
from geopy.geocoders import Nominatim
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [18]:

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<p><b font size=3>7. Explore Toronto Neighbourhoods</font></b></p>
<p>Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.</p>

In [61]:
# The code was removed by Watson Studio for sharing.

Initialized Foursquare credentials: client_id, client_secret, and version in this cell.


In [20]:
toronto_df.loc[0, 'Neighbourhood']

'Lawrence Park'

Get the neighbourhood's Latitude and Longitude

In [21]:
neighbourhood_latitude = toronto_df.loc[0, 'Latitude'] # neighbourhood latitude value
neighbourhood_longitude = toronto_df.loc[0, 'Longitude'] # neighbourhood longitude value

neighbourhood_name = toronto_df.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Lawrence Park are 43.7280205, -79.3887901.


<B>Geting top 100 venues that are in Lawrence Park within a radius of 500 meters.<B>

In [63]:
search_query = 'Lawrence Park'
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    VERSION, 
    search_query, 
    radius, LIMIT)
print("Created url with Foursquare credentials:'https://api.foursquare.com/v2/venues/search?client_id=*******&client_secret=************&ll=43.7280205,-79.3887901&v=20180605&query=Lawrence Park&radius=500&limit=100'")

Created url with Foursquare credentials:'https://api.foursquare.com/v2/venues/search?client_id=*******&client_secret=************&ll=43.7280205,-79.3887901&v=20180605&query=Lawrence Park&radius=500&limit=100'


In [25]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c5e0779dd5797192ad688e7'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d163941735',
      'name': 'Park',
      'pluralName': 'Parks',
      'primary': True,
      'shortName': 'Park'}],
    'hasPerk': False,
    'id': '50e6da19e4b0d8a78a0e9794',
    'location': {'address': '3055 Yonge Street',
     'cc': 'CA',
     'city': 'Toronto',
     'country': 'Canada',
     'crossStreet': 'Lawrence Avenue East',
     'distance': 465,
     'formattedAddress': ['3055 Yonge Street (Lawrence Avenue East)',
      'Toronto ON',
      'Canada'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.72696303913755,
       'lng': -79.39438246708775}],
     'lat': 43.72696303913755,
     'lng': -79.39438246708775,
     'state': 'ON'},
    'name': 'Lawrence Park Ravine',
    'referralId': 'v-1549666169'},
   {'categories': [{'ico

In [26]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [33]:

venues = results['response']['venues'] 
nearby_venues = json_normalize(venues) # flatten JSON

filtered_columns = ['name', 'categories', 'location.lat', 'location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis=1)
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Lawrence Park Ravine,Park,43.726963,-79.394382
1,TTC Bus #162 - Lawrence-Donway,Bus Line,43.728026,-79.382805
2,Wanless Park,Neighborhood,43.727064,-79.394406


In [29]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare.


<b>Explore all neighbourhoods in Toronto</b>

In [34]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
toronto_venues = getNearbyVenues(names=toronto_df['Neighbourhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Business Reply Mail Processing Centre 969 Eastern
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Villag

In [35]:
print(toronto_venues.shape)
toronto_venues.head()

(1694, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,Dim Sum Deluxe,43.726953,-79.394260,Dim Sum Restaurant
2,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
3,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
4,Davisville North,43.712751,-79.390197,Sherwood Park,43.716551,-79.387776,Park


<b>Check how many venues are returned for each neighbourhood</b>

In [36]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
"Brockton,Exhibition Place,Parkdale Village",19,19,19,19,19,19
Business Reply Mail Processing Centre 969 Eastern,19,19,19,19,19,19
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",14,14,14,14,14,14
"Cabbagetown,St. James Town",47,47,47,47,47,47
Central Bay Street,83,83,83,83,83,83
"Chinatown,Grange Park,Kensington Market",100,100,100,100,100,100
Christie,15,15,15,15,15,15


In [37]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 235 uniques categories.


<B>Analyze each Neighbourhood in Toronto</B>

In [38]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Lawrence Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Lawrence Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Lawrence Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Lawrence Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Davisville North,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
toronto_onehot.shape

(1694, 236)

<b>Find mean of occurence of each category for each Neighbourhood in Toronto.</b>

In [40]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,...,0.00000,0.00,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.01,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,"Brockton,Exhibition Place,Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.052632
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.000000,0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
5,"Cabbagetown,St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021277,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
6,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012048,...,0.00000,0.00,0.012048,0.000000,0.012048,0.000000,0.012048,0.000000,0.00,0.012048
7,"Chinatown,Grange Park,Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.050000,0.000000,0.000000,0.050000,0.010000,0.000000,0.00,0.000000
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,Church and Wellesley,0.011628,0.011628,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011628,...,0.00000,0.00,0.000000,0.011628,0.000000,0.011628,0.000000,0.011628,0.00,0.011628


<b>Print each Neighbourhood with top five venues.</b>

In [41]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2  American Restaurant  0.04
3      Thai Restaurant  0.04
4           Steakhouse  0.04


----Berczy Park----
            venue  freq
0     Coffee Shop  0.07
1      Restaurant  0.05
2    Cocktail Bar  0.05
3  Farmers Market  0.04
4     Cheese Shop  0.04


----Brockton,Exhibition Place,Parkdale Village----
            venue  freq
0  Breakfast Spot  0.11
1     Coffee Shop  0.11
2            Café  0.11
3             Gym  0.05
4             Bar  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.11
1          Restaurant  0.05
2          Smoke Shop  0.05
3         Pizza Place  0.05
4                 Spa  0.05


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
              venue  freq
0  Airport Terminal  0.14
1    Airport Lounge  0.14
2   Airport Ser

In [42]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

<b>Display top 10 most common venues of each Neighbourhood.</b>

In [43]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Clothing Store,Asian Restaurant,Restaurant,Hotel,Bar
1,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Café,Beer Bar,Italian Restaurant,Bakery,Steakhouse,Seafood Restaurant,Cheese Shop
2,"Brockton,Exhibition Place,Parkdale Village",Breakfast Spot,Coffee Shop,Café,Performing Arts Venue,Burrito Place,Stadium,Bar,Caribbean Restaurant,Italian Restaurant,Climbing Gym
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Recording Studio,Smoke Shop,Skate Park,Brewery,Burrito Place,Butcher,Restaurant,Comic Shop
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport Service,Airport Terminal,Plane,Harbor / Marina,Airport,Airport Food Court,Airport Gate,Boutique,Boat or Ferry


<b>8. Cluster Neighbourhoods of Toronto</b>

In [44]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [50]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighbourhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Swim School,Dim Sum Restaurant,Bus Line,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Dance Studio,Breakfast Spot,Burger Joint,Food & Drink Shop,Clothing Store,Hotel,Sandwich Place,Park,Gym,Event Space
2,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,0,Clothing Store,Sporting Goods Shop,Coffee Shop,Yoga Studio,Chinese Restaurant,Dessert Shop,Rental Car Location,Diner,Salon / Barbershop,Sandwich Place
3,M4S,Central Toronto,Davisville,43.704324,-79.388790,0,Dessert Shop,Sandwich Place,Pizza Place,Seafood Restaurant,Restaurant,Café,Coffee Shop,Sushi Restaurant,Italian Restaurant,Farmers Market
4,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160,1,Playground,Park,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


<b>Cluster 1</b>

In [56]:
tcluster1 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
tcluster1

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,0,Park,Swim School,Dim Sum Restaurant,Bus Line,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
1,Central Toronto,0,Dance Studio,Breakfast Spot,Burger Joint,Food & Drink Shop,Clothing Store,Hotel,Sandwich Place,Park,Gym,Event Space
2,Central Toronto,0,Clothing Store,Sporting Goods Shop,Coffee Shop,Yoga Studio,Chinese Restaurant,Dessert Shop,Rental Car Location,Diner,Salon / Barbershop,Sandwich Place
3,Central Toronto,0,Dessert Shop,Sandwich Place,Pizza Place,Seafood Restaurant,Restaurant,Café,Coffee Shop,Sushi Restaurant,Italian Restaurant,Farmers Market
5,Central Toronto,0,Coffee Shop,Pub,Pizza Place,American Restaurant,Convenience Store,Sports Bar,Bagel Shop,Supermarket,Sushi Restaurant,Fried Chicken Joint
8,Central Toronto,0,Coffee Shop,Sandwich Place,Café,Pizza Place,History Museum,BBQ Joint,Jewish Restaurant,Vegetarian / Vegan Restaurant,Park,Liquor Store
10,Downtown Toronto,0,Coffee Shop,Restaurant,Park,Italian Restaurant,Bakery,Café,Pub,Pizza Place,Pharmacy,Snack Place
11,Downtown Toronto,0,Japanese Restaurant,Sushi Restaurant,Coffee Shop,Gay Bar,Restaurant,Burger Joint,Café,Pub,Men's Store,Mediterranean Restaurant
12,Downtown Toronto,0,Coffee Shop,Bakery,Café,Pub,Park,Theater,Mexican Restaurant,Breakfast Spot,Restaurant,Electronics Store
13,Downtown Toronto,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Tea Room,Japanese Restaurant,Italian Restaurant,Diner,Pizza Place


<b>Cluster 2</b>

In [52]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,1,Playground,Park,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
9,Downtown Toronto,1,Park,Playground,Trail,Yoga Studio,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


<b>Cluster 3</b>

In [53]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Central Toronto,2,Garden,Yoga Studio,Dog Run,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


<b>Cluster 4</b>

In [54]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Central Toronto,3,Park,Trail,Sushi Restaurant,Jewelry Store,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


<b>Cluster 5</b>

In [55]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,East Toronto,4,Coffee Shop,Pub,Neighborhood,Yoga Studio,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


<b>9. Conclusion</b>

<p>There are 4 boroughs and 38 neighbourhoods in Toronto.</p>
<p>There are 1694 venues in Toronto Neighbourhood.</p>
<p>There are 235 uniques categories in venues.</p>
<p>Cluster 1 has 34 neighbourhoods with common venue categories coffe shop and restaurant/store.</p>
<p>Cluster 2 has 2 neighbourhoods with common venue categories of park and playground.</p>
<p>Cluster 3 has 1 neighbourhoods with venue categories of Garden, Yoga studio and others.</p>
<p>Cluster 4 has 1 neighbourhood  with venue category park, trail and others.</p>
<p>Cluster 5 has 1 neighbourhood   with venue category cofeeshop, pub and others.</p>